In [1]:
!pip install pygame
!pip install svgpathtools

In [2]:
import pygame
from svgpathtools import svg2paths
from numpy import cos, sin, pi, sqrt, arctan2
import pygame.draw as draw

pygame 2.5.2 (SDL 2.28.3, Python 3.10.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def fourier_term(signals, k):
  N = len(signals)
  X_k = complex()
  for n in range(N):
    theta = 2 * pi * k * n / N
    X_k += complex(cos(theta), -sin(theta)) * signals[n]
  return X_k / N

def fourier_transform(signals):  
  return [fourier_term(signals, k) for k in range(len(signals))]

In [4]:
class Wave:
  def __init__(self, frequency, amplitude, phase):
    self.frequency = frequency
    self.amplitude = amplitude
    self.phase = phase
  def __repr__(self):
    return f"Wave(freq={self.frequency}, amp={self.amplitude}, phase={self.phase})"

  @staticmethod
  def from_fourier_term(k, cmplx_num):
    r = cmplx_num.real
    i = cmplx_num.imag
    return Wave(k, sqrt(r * r + i * i), arctan2(i, r))

def generate_wave_info(signals):
  fourier_terms = fourier_transform(signals)
  return [Wave.from_fourier_term(k, term) for k, term in enumerate(fourier_terms)]

In [5]:
def read_file(filename):
  file = open(filename, "r")
  return [[float(num) for num in line.split(", ")] for line in file]

In [6]:
drawing = read_file('assets/capitalE.txt')
xs = [point[0] * 2 for point in drawing]
ys = [point[1] * 2 for point in drawing]

In [11]:
# Load SVG file
paths, attributes = svg2paths('assets/R.svg')
xsR = []
ysR = []

# Extracting coordinates from paths
for path in paths:
    for segment in path:
        # Extract coordinates from the segment
        for point in segment:
            x, y = point.real, point.imag
            xsR.append(x)
            ysR.append(y)

In [12]:
paths, attributes = svg2paths('assets/A.svg')
xsA = []
ysA = []

# Extracting coordinates from paths
for path in paths:
    for segment in path:
        # Extract coordinates from the segment
        for point in segment:
            x, y = point.real, point.imag
            xsA.append(x)
            ysA.append(y)

In [13]:
x_waves = sorted(generate_wave_info(xsA + [x + 100 for x in xsR]), key=lambda x: -x.amplitude)
y_waves = sorted(generate_wave_info(ysA + ysR), key=lambda x: -x.amplitude)

In [9]:
class Point:
  def __init__(self, x, y):
    self.x = x
    self.y = y
  
  def __add__(self, other):
    return Point(self.x + other.x, self.y + other.y)
  
  def __sub__(self, other):
    return Point(self.x - other.x, self.y - other.y)
  
  def __iadd__(self, other):
    self.x += other.x
    self.y += other.y
    return self

  def __truediv__(self, num):
    return Point(self.x / num, self.y / num)
  
  def __mul__(self, num):
    return Point(self.x * num, self.y * num)
  
  def __repr__(self):
    return f"({self.x}, {self.y})"
  
  def as_tuple(self):
    return (self.x, self.y)

In [14]:
# pygame setup
pygame.init()
screensize = Point(1280, 720)
screencenter = screensize / 2
screen = pygame.display.set_mode(screensize.as_tuple())
clock = pygame.time.Clock()
running = True
time = 0
points = []
oldPoints = []
epsilon = pi / 8

while running:
    # poll for events
    # pygame.QUIT event means the user clicked X to close your window
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    # fill the screen with a color to wipe away anything from last frame
    screen.fill("black")

    currPointX = Point(screencenter.x, 100)
    for wave in x_waves:
        prevPoint = currPointX

        freq = wave.frequency
        radius = wave.amplitude
        phase = wave.phase

        offsetPoint = Point(cos(freq * time + phase), sin(freq * time + phase)) * radius
        currPointX = prevPoint + offsetPoint
        draw.line(screen, 'white', prevPoint.as_tuple(), currPointX.as_tuple(), 2)
        radiusPoint = Point(radius, radius)
        topLeft = prevPoint - radiusPoint
        draw.circle(screen, (100, 100, 100), prevPoint.as_tuple(), radius, width = 2)
    
    currPointY = Point(100, screencenter.y)
    for wave in y_waves:
        prevPoint = currPointY

        freq = wave.frequency
        radius = wave.amplitude
        phase = wave.phase

        offsetPoint = Point(cos(freq * time + phase + (pi / 2)), sin(freq * time + phase + (pi / 2))) * radius
        currPointY = prevPoint + offsetPoint
        draw.line(screen, 'white', prevPoint.as_tuple(), currPointY.as_tuple(), 2)
        radiusPoint = Point(radius, radius)
        topLeft = prevPoint - radiusPoint
        draw.circle(screen, (100, 100, 100), prevPoint.as_tuple(), radius, width = 2)

    time += 2 * pi / len(x_waves)
        
    finalPoint = (currPointX.x, currPointY.y)
    draw.line(screen, 'red', currPointX.as_tuple(), finalPoint)
    draw.line(screen, 'red', currPointY.as_tuple(), finalPoint)
    
    points.append((currPointX.x, currPointY.y))
    if (time >= 2 * pi - epsilon):
        oldPoints.append(points[0])
        points = points[1:]
    if len(oldPoints) >= 250:
        oldPoints = oldPoints[1:]

    if len(oldPoints) > 1:
        draw.lines(screen, (100, 100, 100), False, oldPoints)
    if len(points) > 1:
        draw.lines(screen, 'white', False, points)
    
    # flip() the display to put your work on screen
    pygame.display.flip()

    clock.tick(60)  # limits FPS to 60

pygame.quit()

KeyboardInterrupt: 

: 